In [22]:
savedmode_name="model_seed2023_score0.988.pth"

In [23]:
# %pip install --force-reinstall /kaggle/input/unidecode/Unidecode-1.3.8-py3-none-any.whl

In [24]:
# TODO:change path
from cfg import *
from util import *
from model1 import NLPModel
import gc

# Config.data_path='/kaggle/input/pii-detection-removal-from-educational-data/'
# Config.modelsavepath='/kaggle/input/deberbase0213'
tokenizer_path=Config.modelsavepath
modelpath=Config.modelsavepath+savedmode_name
modelpath

'/home/z1933/workplace/machinelearning/PII-Data-Detection//modelsave/model_seed2023_score0.988.pth'

In [25]:
Config.modelsavepath

'/home/z1933/workplace/machinelearning/PII-Data-Detection//modelsave/'

In [26]:
import json
from itertools import chain
from transformers import AutoModelForTokenClassification

In [27]:
data = json.load(open(Config.data_path+"/test.json"))
ds = Dataset.from_dict({
    "full_text": [x["full_text"] for x in data],
    "document": [x["document"] for x in data],
    "tokens": [x["tokens"] for x in data],
    "trailing_whitespace": [x["trailing_whitespace"] for x in data],
})
ds

Dataset({
    features: ['full_text', 'document', 'tokens', 'trailing_whitespace'],
    num_rows: 10
})

In [28]:

tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)
idlabel= json.load(open(Config.modelsavepath+"//idlabel.json"))
id2label=idlabel['id2label']
label2id=idlabel['label2id']
id2label

{'0': 'B-EMAIL',
 '1': 'B-ID_NUM',
 '2': 'B-NAME_STUDENT',
 '3': 'B-PHONE_NUM',
 '4': 'B-STREET_ADDRESS',
 '5': 'B-URL_PERSONAL',
 '6': 'B-USERNAME',
 '7': 'I-ID_NUM',
 '8': 'I-NAME_STUDENT',
 '9': 'I-PHONE_NUM',
 '10': 'I-STREET_ADDRESS',
 '11': 'I-URL_PERSONAL',
 '12': 'O'}

In [29]:
# 如果多次没击中，大概率有问题
preprocesssed_ds=ds.map(preprocesss, fn_kwargs={'tokenizer':tokenizer,'label2id':label2id,'if_train':False},num_proc=8,desc="prepocessing data")
preprocesssed_ds

prepocessing data (num_proc=8):   0%|          | 0/10 [00:00<?, ? examples/s]

Dataset({
    features: ['full_text', 'document', 'tokens', 'trailing_whitespace', 'splited_sens', 'berttokenpos2orgtokenpos', 'bertlabels', 'berttokenids', 'berttokenmask', 'berttokentoken_type_ids'],
    num_rows: 10
})

In [30]:
# rebuid dataset 
tmp_pd=expanddataset(preprocesssed_ds)
print(tmp_pd['berttokenids'].str.len().agg(['mean','max','std']))
tmp_pd

mean     29.732601
max     541.000000
std      54.569083
Name: berttokenids, dtype: float64


,full_text,document,tokens,trailing_whitespace,splited_sens,berttokenpos2orgtokenpos,berttokenids,berttokenmask,berttokentoken_type_ids,bertlabels
0,design thinking for innovation reflexion-avril...,7,"[design, thinking, for, innovation, reflexion,...","[True, True, True, True, False, False, True, F...",[design thinking for innovation reflexion-avri...,"[-1, 0, 1, 2, 3, 4, 4, 5, 6, 6, 7, 8, 9, 9, 9,...","[1, 587, 1291, 270, 3513, 28310, 4593, 271, 14...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-100, -100, -100, -100, -100, -100, -100, -10..."
1,design thinking for innovation reflexion-avril...,7,"[design, thinking, for, innovation, reflexion,...","[True, True, True, True, False, False, True, F...",[design thinking for innovation reflexion-avri...,"[-1, 12, 13, 14, -1]","[1, 1719, 429, 1857, 2]","[1, 1, 1, 1, 1]","[0, 0, 0, 0, 0]","[-100, -100, -100, -100, -100]"
2,design thinking for innovation reflexion-avril...,7,"[design, thinking, for, innovation, reflexion,...","[True, True, True, True, False, False, True, F...",[design thinking for innovation reflexion-avri...,"[-1, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 2...","[1, 262, 1637, 584, 380, 264, 408, 305, 6998, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-100, -100, -100, -100, -100, -100, -100, -10..."
3,design thinking for innovation reflexion-avril...,7,"[design, thinking, for, innovation, reflexion,...","[True, True, True, True, False, False, True, F...",[design thinking for innovation reflexion-avri...,"[-1, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 5...","[1, 339, 1444, 269, 266, 791, 2269, 302, 970, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-100, -100, -100, -100, -100, -100, -100, -10..."
4,design thinking for innovation reflexion-avril...,7,"[design, thinking, for, innovation, reflexion,...","[True, True, True, True, False, False, True, F...",[design thinking for innovation reflexion-avri...,"[-1, 110, 111, 112, 113, 114, 115, -1]","[1, 291, 1637, 303, 386, 5228, 294, 2]","[1, 1, 1, 1, 1, 1, 1, 1]","[0, 0, 0, 0, 0, 0, 0, 0]","[-100, -100, -100, -100, -100, -100, -100, -100]"
...,...,...,...,...,...,...,...,...,...,...
268,gandhi institute of technology and management ...,123,"[gandhi, institute, of, technology, and, manag...","[True, True, True, True, True, True, False, Tr...",[gandhi institute of technology and management...,"[-1, 1600, 1601, 1602, 1602, 1602, 1603, 1603,...","[1, 453, 260, 263, 368, 12626, 4402, 260, 2980...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-100, -100, -100, -100, -100, -100, -100, -10..."
269,gandhi institute of technology and management ...,123,"[gandhi, institute, of, technology, and, manag...","[True, True, True, True, True, True, False, Tr...",[gandhi institute of technology and management...,"[-1, 1621, 1621, 1622, 1623, 1623, 1624, 1624,...","[1, 66641, 358, 261, 4402, 260, 2982, 474, 513...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-100, -100, -100, -100, -100, -100, -100, -10..."
270,gandhi institute of technology and management ...,123,"[gandhi, institute, of, technology, and, manag...","[True, True, True, True, True, True, False, Tr...",[gandhi institute of technology and management...,"[-1, 1651, 1652, 1653, 1653, 1654, 1654, 1655,...","[1, 456, 260, 2994, 45454, 865, 260, 261, 507,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-100, -100, -100, -100, -100, -100, -100, -10..."
271,gandhi institute of technology and management ...,123,"[gandhi, institute, of, technology, and, manag...","[True, True, True, True, True, True, False, Tr...",[gandhi institute of technology and management...,"[-1, 1670, 1671, 1672, 1673, 1674, 1674, 1675,...","[1,

In [31]:
full_ds=Dataset.from_pandas(tmp_pd)
full_ds

Dataset({
    features: ['full_text', 'document', 'tokens', 'trailing_whitespace', 'splited_sens', 'berttokenpos2orgtokenpos', 'berttokenids', 'berttokenmask', 'berttokentoken_type_ids', 'bertlabels'],
    num_rows: 273
})

In [32]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model=NLPModel(id2label,label2id,Config.modelsavepath,training=False).to(device)
model.load_state_dict(torch.load(modelpath))

<All keys matched successfully>

In [33]:
data_collator = Collate(tokenizer=tokenizer)
val_dataloader=DataLoader(full_ds,batch_size=1,pin_memory=True,collate_fn=data_collator)

In [34]:
model.eval()

allprecitions=[]
for step,dataset in enumerate(tqdm(val_dataloader)):

    with torch.no_grad():
        ids = dataset["ids"].to(device,non_blocking=True)
        mask = dataset["mask"].to(device,non_blocking=True)
        tokentype = dataset["type_ids"].to(device,non_blocking=True)
        logit,loss = model(ids,mask,tokentype)
        allprecitions.append(logit2truepredic(logit))
        del ids,mask,tokentype

gc.collect()
torch.cuda.empty_cache()

100%|██████████| 273/273 [00:04<00:00, 60.15it/s]


In [35]:
triplets = set()
document, token_ids, labels, token_strs = [], [], [], []

idx=0
for  token_map, tokens, doc in zip(full_ds["berttokenpos2orgtokenpos"], full_ds["tokens"], full_ds["document"]):
    prediction=allprecitions[idx]
    for i in range(0,len(prediction)):
        if token_map[i]==-1: # no position map
            continue
        label_pred = id2label[str(prediction[i])]
        token_id=token_map[i]
        token_str=tokens[token_id]
        if label_pred != "O":
            triplet=(doc,token_id,prediction[i]) # 多对一映射，只记录第一个的预测
            if triplet not in triplets:
                document.append(doc)
                token_ids.append(token_id)
                labels.append(label_pred)
                token_strs.append(token_str)
                triplets.add(triplet)
    idx+=1

In [36]:
token_strs

['nathalie',
 'sylla',
 'nathalie',
 'sylla',
 'nathalie',
 'sylla',
 'diego',
 'estrada',
 'diego',
 'estrada',
 'gilberto',
 'gamboa',
 'sindy',
 'samaca',
 'nadine',
 'born',
 'eladio',
 'amaya',
 'silvia',
 'villalobos',
 'sakir',
 'ahmad',
 'francisco',
 'ferreira',
 'stefano',
 'lovato']

In [37]:
df = pd.DataFrame({
    "document": document,
    "token": token_ids,
    "label": labels,
    "token_str": token_strs
})
df["row_id"] = list(range(len(df)))
display(df.head(100))

,document,token,label,token_str,row_id
0,7,9,B-NAME_STUDENT,nathalie,0
1,7,10,I-NAME_STUDENT,sylla,1
2,7,482,B-NAME_STUDENT,nathalie,2
3,7,483,I-NAME_STUDENT,sylla,3
4,7,741,B-NAME_STUDENT,nathalie,4
5,7,742,I-NAME_STUDENT,sylla,5
6,10,0,B-NAME_STUDENT,diego,6
7,10,1,I-NAME_STUDENT,estrada,7
8,10,464,B-NAME_STUDENT,diego,8
9,10,465,I-NAME_STUDENT,estrada,9


In [38]:
df[["row_id", "document", "token", "label"]].to_csv("submission.csv", index=False)